# Two Sources : Internal Bleed

In [1]:
import numpy as np
import os
import librosa as lb
import soundfile as sf
import museval
from matplotlib import pyplot as plt
from tqdm import tqdm
import librosa.display

In [6]:
def random_gen():
    k = np.round(np.random.uniform(0.05, 0.31), 3)
    return k
def random_diag():
    k = np.round(np.random.uniform(0.6, 1), 3)
    return k

def gen_A():
    A = np.array([[random_diag(), random_gen()],
                  [random_gen(), random_diag()]])
    return np.round(A, 2)

In [7]:
def conditions(vx, bx, block):
    flag = True
    if vx.shape[0] != block:
        flag = False
    if np.linalg.norm(vx) <= 0.9 or np.linalg.norm(bx) <= 0.9:
        flag = False
    return flag

In [8]:
clean_path = "/home/rajesh/Desktop/Datasets/musdb18hq/test/"
outpath = "/home/rajesh/Desktop/Datasets/musdb18hq_2source/test/"

In [9]:
files = sorted(os.listdir(clean_path))

for file in tqdm(files):
    vocal, fs = lb.load(clean_path+file+"/vocals.wav")
    bass, fs = lb.load(clean_path+file+"/bass.wav")
    
    path = outpath+file
    os.makedirs(path, exist_ok=True)
    os.makedirs(path+'/Audio', exist_ok=True)
    os.makedirs(path+'/Numpy', exist_ok=True)
    A = gen_A()
    np.save(path+'/mixing.npy', A)
    
    sec = 10
    block = sec*fs
    length = vocal.shape[0]
    for i in range(0, length, block):
        vx = vocal[i:i+block]
        bx = bass[i:i+block]

        if conditions(vx, bx, block):

            fapath = path+'/Audio/{}_{}/'.format(int(i/fs), int(i/fs)+sec)
            fnpath = path+'/Numpy/{}_{}/'.format(int(i/fs), int(i/fs)+sec)
            os.makedirs(fapath, exist_ok=True)
            os.makedirs(fnpath, exist_ok=True)

            np.save(fnpath+'vocal.npy', vx)
            np.save(fnpath+'bass.npy', bx)

            s = np.array([vx, bx])
            x = A @ s

            np.save(fnpath+'bvocal.npy', x[0])
            np.save(fnpath+'bbass.npy', x[1])

            sf.write(fapath+'vocal.wav', vx, fs)
            sf.write(fapath+'bass.wav', bx, fs)

            sf.write(fapath+'bvocal.wav', x[0], fs)
            sf.write(fapath+'bbass.wav', x[1], fs)

100%|███████████████████████████████████████████| 50/50 [09:48<00:00, 11.76s/it]


In [10]:
path = "/home/rajesh/Desktop/Datasets/musdb18hq_2source/test/"

In [11]:
mainfolder = sorted(os.listdir(path))

In [12]:
xtrain, ytrain = [], []
for folder in tqdm(mainfolder[0:50]):
    dept_path = path+folder+'/Numpy/'
    subfolder = sorted(os.listdir(dept_path))
    for files in subfolder:
        v = np.load(dept_path+files+'/vocal.npy')
        b = np.load(dept_path+files+'/bass.npy')
        
        bv = np.load(dept_path+files+'/bvocal.npy')
        bb = np.load(dept_path+files+'/bbass.npy')
        
        S = np.array([v, b])
        X = np.array([bv, bb])
        xtrain.append(X)
        ytrain.append(S)
        
np.save(path+'xtest_0_50.npy', xtrain)
np.save(path+'ytest_0_50.npy', ytrain)

100%|███████████████████████████████████████████| 50/50 [01:49<00:00,  2.19s/it]
